In [ ]:
import geopandas as gpd
import pandas as pd
from pyproj import Proj, CRS, Transformer
from shapely.geometry import Point
from shapely import wkt
import matplotlib.pyplot as plt
import folium
from folium import plugins
import branca
import branca.colormap as cm

#plt.rcParams['figure.figsize'] = [8,8]


In [ ]:
data =gpd.read_file("../geojson/agricultural_risk/Agricultural_Critical_Risk_Areas.geojson")
data.head()

In [ ]:
data["GlobalID"].nunique()

In [ ]:
'''
colormap = cm.StepColormap(colors=['green', 'blue', 'red'],
                             index=[0,1,2,3],
                             vmin=0,
                             vmax=3,
                             caption='Risk Score (1=Low, 2=medium, 3=High)')

raw_risk=data[['bh', 'hydoconnec','rainfall_2', 'slope_2', 'landclass2', 'host_2', 'soilp_2', 
              'q10_2','riverbuff2', 'rwb_stat_2', 'erosion', 'riskscore2', 'ruleid','geometry'
              ]]
              
raw_risk.explore("ruleid", cmap=colormap)
''';

In [ ]:
risk_geometry=data["geometry"]

In [ ]:
'''
risk_point=[]
for i in risk_geometry:
    risk_point.append(i.centroid.wkt)

risk_point=gpd.GeoDataFrame(risk_point.copy())
risk_point.columns=["geometry"]
#risk_point["geometry"]=risk_point["geometry"].apply(wkt.loads)
risk_point.set_geometry("geometry", inplace=True)
risk_point
''';

In [ ]:
risk=data[['bh', 'hydoconnec','rainfall_2', 'slope_2', 'landclass2', 'host_2', 'soilp_2', 
              'q10_2','riverbuff2', 'rwb_stat_2', 'erosion', 'riskscore2', 'ruleid'
              ]].copy()

risk_point=gpd.GeoDataFrame(risk_geometry.apply(lambda x:x.centroid.wkt), columns=["geometry"])

risk=gpd.GeoDataFrame(pd.concat([risk.copy(), risk_point], axis=1))
risk["geometry"]=risk["geometry"].apply(wkt.loads)
risk.set_geometry("geometry", inplace=True)

In [ ]:
def create_nimap():
    nimap = folium.Map(
        location=[54.667521, -6.8135], #54°36'27"N, 6°41'35"W
        zoom_start=9
    )
    return nimap

In [ ]:
'''
fig = folium.Figure(width=1200, height=1000)
m=create_nimap()

colormap = cm.StepColormap(colors=['green', 'blue', 'red'],
                             index=[0,1,2,3],
                             vmin=0,
                             vmax=3,
                             caption='Risk Score (1=Low, 2=medium, 3=High)')
              
risk_map=risk.explore("ruleid",m=m, cmap=colormap)
risk_map.add_to(fig)

risk_map.save("risk_map.html")
''';

In [ ]:
surface_ard=pd.read_csv("../jupyter/custom_csv/surface_soil_aqua.csv")
surface_ard['geometry'] = surface_ard['geometry'].apply(wkt.loads)

deep_ard=pd.read_csv("../jupyter/custom_csv/deep_soil_CEC.csv")
deep_ard['geometry'] = deep_ard['geometry'].apply(wkt.loads)

In [ ]:
'''
surface_ard["ARCA_ID"] = surface_ard.apply(lambda p: data[data.apply(lambda g: g.geometry.contains(p.geometry), axis=1)].GlobalID.iloc[:1].reset_index(drop=True), axis=1)
surface_merged_df = pd.merge(surface_ard, data, left_on='ARCA_ID', right_on='GlobalID')
surface_merged_df["geometry"]=surface_merged_df["geometry_x"]
surface_merged_df.drop(["geometry_x","geometry_y"], axis=1, inplace=True)
surface_merged_df.to_csv("surface_merged_df.csv")
'''

In [ ]:
'''
deep_ard["ARCA_ID"] = deep_ard.apply(lambda p: data[data.apply(lambda g: g.geometry.contains(p.geometry), axis=1)].GlobalID.iloc[:1].reset_index(drop=True), axis=1)
deep_merged_df = pd.merge(deep_ard, data, left_on='ARCA_ID', right_on='GlobalID')
deep_merged_df["geometry"]=deep_merged_df["geometry_x"]
deep_merged_df.drop(["geometry_x","geometry_y"], axis=1, inplace=True)
deep_merged_df.to_csv("deep_merged_df.csv")
'''


In [ ]:
surface_merged_df=pd.read_csv("../jupyter/custom_csv/surface_merged_df.csv")
surface_merged_df['geometry'] = surface_merged_df['geometry'].apply(wkt.loads)
deep_merged_df=pd.read_csv("../jupyter/custom_csv/deep_merged_df.csv")
deep_merged_df['geometry'] = deep_merged_df['geometry'].apply(wkt.loads)

In [ ]:
surface_merged_df.head()

In [ ]:
deep_merged_df.head()

In [ ]:
surface_merged_df.columns

In [ ]:
#merged_dff["Ni"]=merged_dff.join(ard,on="Sample",rsuffix='_other').Ni
agri_risk = gpd.GeoDataFrame(surface_merged_df)

In [ ]:
soil_risk=agri_risk[['bh','hydoconnec', 'rainfall_2', 'slope_2',
       'landclass2', 'host_2', 'soilp_2', 'q10_2', 'riverbuff2', 'rwb_stat_2',
       'erosion', 'riskscore2', 'ruleid', 'pH', '%OM', 'geometry']].copy()

In [ ]:
soil_risk

In [ ]:
soil_risk["riverbuff2"].unique()

In [ ]:
soil_risk.groupby(["hydoconnec"])["bh"].count().to_frame()

In [ ]:
risk_hydro=agri_risk[(agri_risk['hydoconnec']== 'Yes') & (agri_risk['ruleid']==3)]
len(risk_hydro)

In [ ]:
colormap = cm.StepColormap(colors=['green', 'blue', 'red'],
                             index=[0,1,2,3],
                             vmin=0,
                             vmax=3,
                             caption='Risk Score (1=Low, 2=medium, 3=High)')

ni_map=create_nimap()
fig = folium.Figure(width=1200, height=1000)

soil_risk_map=soil_risk.explore("ruleid", cmap=colormap, m=ni_map, marker_kwds={"radius": 5})
soil_risk_map.add_to(fig)

In [ ]:
soil_risk['lat']=soil_risk.geometry.y
soil_risk['lng']=soil_risk.geometry.x

soil_risk_low = soil_risk[(soil_risk['ruleid']==1)]
soil_risk_med = soil_risk[(soil_risk['ruleid']==2)]
soil_risk_high = soil_risk[(soil_risk['ruleid']==3)]


In [ ]:
m = create_nimap()
fig = folium.Figure(width=1200, height=1000)

group_1 = folium.FeatureGroup("Low Risk").add_to(m)
for loc, p in zip(zip(soil_risk_low["lat"],soil_risk_low["lng"]),soil_risk_low["ruleid"]):
      folium.CircleMarker(location=loc,
                          radius= 5,
                          fill=True,
                          #fill_opacity=1,
                          color='green'
                          ).add_to(group_1)

group_2 = folium.FeatureGroup("Medium Risk").add_to(m)
for loc, p in zip(zip(soil_risk_med["lat"],soil_risk_med["lng"]),soil_risk_med["ruleid"]):
      folium.CircleMarker(location=loc,
                          radius= 5,
                          fill=True,
                          #fill_opacity=1,
                          color='darkorange').add_to(group_2)

group_3 = folium.FeatureGroup("High Risk").add_to(m)
for loc, p in zip(zip(soil_risk_high["lat"],soil_risk_high["lng"]),soil_risk_high["ruleid"]):
      folium.CircleMarker(location=loc,
                          radius= 5,
                          fill=True,
                          #fill_opacity=1,
                          color='red').add_to(group_3)

plugins.Geocoder().add_to(m)
folium.LayerControl(collapsed = False).add_to(m)

m.add_to(fig)
#m

In [ ]:
surface_CEC=gpd.GeoDataFrame(pd.read_csv("../jupyter/custom_csv/surface_soil_CEC.csv"))
deep_CEC=gpd.GeoDataFrame(pd.read_csv("../jupyter/custom_csv/deep_soil_CEC.csv"))

surface_CEC['geometry'] = surface_CEC['geometry'].apply(wkt.loads)
deep_CEC['geometry'] = deep_CEC['geometry'].apply(wkt.loads)

#surface_CEC=surface_CEC.drop("Sample", axis=1)

In [ ]:
surface_CEC["Sample"].nunique()

In [ ]:
deep_CEC["Sample"].nunique()

In [ ]:
soil_CEC_df=(surface_CEC.copy().rename(columns={'Type':'Surface_Type'}))
soil_CEC_df

In [ ]:
soil_CEC=pd.merge(soil_CEC_df, deep_CEC[["Sample", "Type"]], on='Sample', how='inner')
soil_CEC=soil_CEC.rename(columns={'Type':'Deep_Type'})
soil_CEC

In [ ]:
#for organic surface soil, what is the deep soil type?

surface_organic=soil_CEC.loc[soil_CEC["Surface_Type"]=="Organic Soil", "Deep_Type"].to_frame()
surface_organic_sum=surface_organic.copy().groupby(["Deep_Type"])["Deep_Type"].count().nlargest().to_frame()
surface_organic_sum_percent=surface_organic_sum.apply(lambda x:x/sum(x)*100).astype(int)
surface_organic_sum_percent.columns=["Organic"]

surface_organic_sum_percent

In [ ]:
surface_clay=soil_CEC.loc[soil_CEC["Surface_Type"]=="Clay", "Deep_Type"].to_frame()
surface_clay_sum=surface_clay.copy().groupby(["Deep_Type"])["Deep_Type"].count().nlargest().to_frame()
surface_clay_sum_percent=surface_clay_sum.apply(lambda x:x/sum(x)*100).astype(int)
surface_clay_sum_percent.columns=["Clay"]

surface_clay_sum_percent

In [ ]:
surface_silt=soil_CEC.loc[soil_CEC["Surface_Type"]=="Silt", "Deep_Type"].to_frame()
surface_silt_sum=surface_silt.copy().groupby(["Deep_Type"])["Deep_Type"].count().nlargest().to_frame()
surface_silt_sum_percent=surface_silt_sum.apply(lambda x:x/sum(x)*100).astype(int)
surface_silt_sum_percent.columns=["Silt"]
surface_silt_sum_percent

In [ ]:
surface_loam=soil_CEC.loc[soil_CEC["Surface_Type"]=="Loam", "Deep_Type"].to_frame()
surface_loam_sum=surface_loam.copy().groupby(["Deep_Type"])["Deep_Type"].count().nlargest().to_frame()
surface_loam_sum_percent=surface_loam_sum.apply(lambda x:x/sum(x)*100).astype(int)
surface_loam_sum_percent.columns=["Loam"]
surface_loam_sum_percent

In [ ]:
surface_sand=soil_CEC.loc[soil_CEC["Surface_Type"]=="Sand", "Deep_Type"].to_frame()
surface_sand_sum=surface_sand.copy().groupby(["Deep_Type"])["Deep_Type"].count().nlargest().to_frame()
surface_sand_sum_percent=surface_sand_sum.apply(lambda x:x/sum(x)*100).astype(int)
surface_sand_sum_percent.columns=["Sand"]
surface_sand_sum_percent

In [ ]:
non_organic_deep=surface_organic_sum[surface_organic_sum.index != "Organic Soil"]
non_clay_deep=surface_organic_sum[surface_organic_sum.index != "Clay"]
non_silt_deep=surface_organic_sum[surface_organic_sum.index != "Silt"]
non_loam_deep=surface_organic_sum[surface_organic_sum.index != "Loam"]
non_sand_deep=surface_organic_sum[surface_organic_sum.index != "Sand"]


In [ ]:
surface_organic_deep=soil_CEC.loc[soil_CEC["Surface_Type"]=="Organic Soil", ["Deep_Type", "geometry"]]
non_organic_deep=surface_organic_deep[surface_organic_deep["Deep_Type"]!="Organic Soil"]

surface_clay_deep=soil_CEC.loc[soil_CEC["Surface_Type"]=="Clay", ["Deep_Type", "geometry"]]
non_clay_deep=surface_clay_deep[surface_clay_deep["Deep_Type"]!="Clay"]

surface_silt_deep=soil_CEC.loc[soil_CEC["Surface_Type"]=="Silt", ["Deep_Type", "geometry"]]
non_silt_deep=surface_silt_deep[surface_silt_deep["Deep_Type"]!="Silt"]

surface_loam_deep=soil_CEC.loc[soil_CEC["Surface_Type"]=="Loam", ["Deep_Type", "geometry"]]
non_loam_deep=surface_loam_deep[surface_loam_deep["Deep_Type"]!="Loam"]

surface_sand_deep=soil_CEC.loc[soil_CEC["Surface_Type"]=="Sand", ["Deep_Type", "geometry"]]
non_sand_deep=surface_sand_deep[surface_sand_deep["Deep_Type"]!="Sand"]

In [ ]:
total_deep_soil=pd.concat([surface_organic_sum_percent, 
           surface_clay_sum_percent, 
           surface_silt_sum_percent, 
           surface_loam_sum_percent, 
           surface_sand_sum_percent], 
           axis=1,
           keys=["Surface Soil Type","Surface Soil Type","Surface Soil Type","Surface Soil Type", "Surface Soil Type"]
           ).fillna(0)


total_deep_soil.index.names=["Deep Soil Type"]
total_deep_soil=total_deep_soil.reindex(["Organic Soil", "Clay", "Silt", "Loam", "Sand"])
total_deep_soil=total_deep_soil.copy().astype(int)
total_deep_soil

In [ ]:
soil_bar=total_deep_soil.droplevel(0, axis=1).T
soil_bar

In [ ]:
ax=soil_bar.plot(kind='bar',
              stacked=True,
              xlabel="$\\bf{Surface\ Soil\ Type}$",
              rot=0,
              color=['green','grey','blue','brown', 'gold'], 
              figsize = (10,8),
              title = "$\\bf{Percentage\ of\ Soil\ Type\ in\ Deep\ Soil\ per\ Type\ of\ Surface\ Soil}$"
              )

'''
for c in ax.containers:
    # remove the labels parameter if it's not needed for customized labels
    ax.bar_label(c, fmt='%0.0f',label_type='center', color="white")
'''

plt.legend(loc="upper right", 
                       bbox_to_anchor=(1.2, 1.05),
                       title="$\\bf{Deep\ Soil\ Type}$"
                       );


In [ ]:
soil_risk

In [ ]:
ni_map = create_nimap()
fig = folium.Figure(width=1200, height=1000)
cmap=cm.LinearColormap(
    ['red', 'purple', 'green', 'blue'],
    index=[0, 0.05, 0.1, 0.15],
    vmin=0.00, vmax=0.15
)

rain=soil_risk.explore("rainfall_2", cmap=cmap, m=ni_map, marker_kwds={"radius": 5})
rain.add_to(fig)


In [ ]:
ni_map = create_nimap()
fig = folium.Figure(width=1200, height=1000)
cmap=cm.LinearColormap(
    ['blue', 'purple', 'orange', 'red'],
    index=[0, 0.2, 0.4, 0.6],
    vmin=0.00, vmax=0.6
)

erosion=soil_risk.explore("erosion", cmap=cmap, m=ni_map, marker_kwds={"radius": 5})
erosion.add_to(fig)


In [ ]:
ni_map = create_nimap()

fig = folium.Figure(width=1200, height=1000)
rainfall_colour=cmap=cm.LinearColormap(['red', 'purple', 'green', 'blue'],index=[0, 0.05, 0.1, 0.15],vmin=0.00, vmax=0.15)
erosion_colour=cm.LinearColormap(['blue', 'purple', 'orange', 'red'],index=[0, 0.2, 0.4, 0.6],vmin=0.00, vmax=0.6)

m=soil_risk.explore(column="rainfall_2", m=ni_map, cmap=rainfall_colour, show=False, legend=False, name="Rainfall", marker_kwds={"radius": 5})
soil_risk.explore(column="erosion", m=m, cmap=erosion_colour, show=False, legend=False, name="Erosion", marker_kwds={"radius": 5})


folium.TileLayer('stamenterrain', name="Terrain").add_to(m)
folium.TileLayer('cartodbpositron', name="Greyscale").add_to(m)
folium.TileLayer('cartodbdark_matter', name="Dark").add_to(m)
plugins.Geocoder().add_to(m)
folium.LayerControl(collapsed=False).add_to(m)  # use folium to add layer control


m.add_to(fig)



In [ ]:
ni_map = create_nimap()
fig = folium.Figure(width=1200, height=1000)
cmap=cm.LinearColormap(
    ['red', 'purple', 'green', 'blue'],
    index=[0, 0.05, 0.1, 0.15],
    vmin=0.00, vmax=0.15
)

rain=soil_risk.explore("rainfall_2", cmap=cmap, m=ni_map, marker_kwds={"radius": 5})
rain.add_to(fig)



ni_map = create_nimap()
fig = folium.Figure(width=1200, height=1000)
cmap=cm.LinearColormap(
    ['blue', 'purple', 'orange', 'red'],
    index=[0, 0.2, 0.4, 0.6],
    vmin=0.00, vmax=0.6
)

erosion=soil_risk.explore("erosion", cmap=cmap, m=ni_map, marker_kwds={"radius": 5})
erosion.add_to(fig)


